# License Management

At 1.8.5 and the upcoming 1.9.0 release the ArcGIS API for Python has many management task improvements.  In this demo we will show how to manage user licenses on ArcGIS Online.

#### Setup the Environment

In [ ]:
from IPython import display
import pandas as pd
from arcgis.gis import GIS

In [ ]:
gis = GIS(profile='your_online_profile')

#### Access the License Manager

In [ ]:
lm = gis.admin.license

In [ ]:
licenses = lm.all()

In [ ]:
gathered = []
for lic in licenses:
    try:
        stats = lic.all()
        for s in stats:
            s['entitlements'] = ",".join(s['entitlements'])
        gathered.extend(stats)
    except:
        ...

In [ ]:
df = pd.DataFrame(gathered)

In [ ]:
df.head()

#### Create a Pandas DataFrame for Easy Visualization of Data

In [ ]:
groupby = pd.DataFrame(gathered).groupby('username')
q = df.lastLogin == -1
df.loc[q, ['lastLogin']] = None
df.lastLogin = pd.to_datetime(df.lastLogin, unit='ms')

In [ ]:
exploded_df = (df
               .copy()
               .set_index(df.columns.drop('entitlements',1).tolist())
               .entitlements.str.split(',', expand=True)
               .stack()
               .reset_index()
               .rename(columns={0:'entitlements'})
               .loc[:, df.columns]
)

In [ ]:
grouped_df = exploded_df.groupby('entitlements')
for key, item in grouped_df:
    print(f"entitlement: {key}")
    display.display(grouped_df.get_group(key))

#### Total Entitlement and Extension Count

In [ ]:
exploded_df.groupby('username').username.count()